# News text data cleaning

### We are going to explore ways to clean text data, namely to remove unncessary sentences initiated by "baca juga" and "advertisement" strings. In this document we will explore several means of text cleaning, such as text segmentation using spaCy and NLTK, and by using heuristic approach to develop algorithm that cleans text data from aforementioned strings.

## 1. Reading dataset

#### We will be reading a dataset/corpus containing news text data.

In [71]:
import pandas as pd
import os

# reding the news dataset
news_dataset = pd.read_csv(f"{os.getcwd()}/news_dataset.csv")
news_dataset_clean = news_dataset.dropna(subset=['content']) # drop na values
news_dataset_clean['content'].drop_duplicates().reset_index(drop=True) # drop duplicate content
news_dataset_clean = news_dataset_clean.reset_index(drop=True)

print(news_dataset_clean["content"])

0      ALASKA — It's been snowing a lot lately in nor...
1      ESB Science Blast. 2.508 suka. An incredible S...
2      Suara.com - BRIZZI merupakan salah satu kartu ...
3      - Cash Tree Siapa sih yang tidak kenal dengan ...
4      KBEonline.id – Solo Leveling Season 2 episode ...
                             ...                        
897    HAIJAKARTA.ID – Nomor HP kamu bisa menerima sa...
898    JABAR EKSPRES – Hadir kembali link penghasil s...
899    Jakarta, benang.id – GoPay, unit bisnis Financ...
900    SINGAPORE: The Competition and Consumer Commis...
901    Once you sign up, $111 is added to your bonus ...
Name: content, Length: 902, dtype: object


#### The dataset contains 902 rows of news texts.

In [72]:
import warnings

warnings.simplefilter("ignore", UserWarning)

# regex with the unecessary keywords
regex_unecessary_words = r"(?i)\b(baca juga|advertisement|iklan)\b"

# add new column matches where True indicates instances of news text with unecessary word
news_dataset_clean['matches'] = news_dataset_clean['content'].str.contains(regex_unecessary_words)

# count the no. of True observation of column 'matches' and len of news_dataset_clean
nrow_matches = news_dataset_clean['matches'].sum()
nrow_news_dataset = len(news_dataset_clean)

print(f"{nrow_matches} instances of news text with unecessary words \n{nrow_matches/nrow_news_dataset:.2f} percent of instances with unecessary words")

310 instances of news text with unecessary words 
0.34 percent of instances with unecessary words


#### From 902 instances, there are 310 instances (34%) of news text containing unecessary words. Below are the randomly picked news text containing aforementioned words:

In [73]:
import random
import re

# the keywords inside the regex for loop
key_unecessary_words = ["baca juga", "advertisement", "iklan"]

# looping each keyword in key_unecessary_words
for keyword in key_unecessary_words:
    matches = news_dataset_clean[news_dataset_clean['content'].str.contains(r'(?i)\b' + re.escape(keyword) + r'\b', na=False)]
    matches = matches.reset_index(drop=True) # reset the indexing
    print(f"--- Unecessary words: {keyword} ({len(matches)} observations) ---")

    if len(matches) > 0:
        for num in range(min(3, len(matches))):
            random_news_instance = random.randint(0, len(matches) - 1)
            print(f"Random news {num}: {matches['content'].iloc[random_news_instance]}\n")
    else:
        print(f"No matches for {keyword} found in the dataset.\n")

--- Unecessary words: baca juga (291 observations) ---
Random news 0: tirto.id - Pembayaran digital terbaru bernama QRIS TAP telah diresmikan oleh Bank Indonesia pada Jumat (14/3/2025) di Jakarta Pusat. Tercatat ada sejumlah 15 penyedia jasa pembayaran (PJP) berupa bank dan e-wallet yang dapat menerapkan fitur tersebut saat ini. QRIS TAP adalah pengembangan dari metode QRIS biasa, tetapi telah berinovasi menggunakan teknologi berbasis Near Field Communication (NFC). Untuk menerapkan metode pembayaran tersebut, pengguna harus memiliki ponsel berbasis NFC. Teknologi tersebut memungkinkan pengguna untuk tidak melakukan pemindaian kode QR (QRIS CPM) lagi. Cukup menempelkan atau mendekatkan ponsel ke NFC Reader, lalu sistem yang tersedia akan membaca data dari e-wallet atau e-banking milik pengguna. Proses pembacaan data dari ponsel NFC oleh NFC Reader diklaim hanya memakan waktu 0,3 detik saja. Ini jauh lebih cepat dibanding QRIS dengan basis teknologi chip yang butuh 4-5 detik untuk memin

#### Using regular expression, it is detected that there are 310 instances of news texts containing "baca juga", "advertisement", or "iklan" strings. We can also see the random news text containing the mentioned strings through regular expression (regex). However it is important to take these into consideration:
1. The regex retrieves solely news with abovementioned strings; it does not take the sentence where the strings are bound into consideration (e.g., the "baca juga" strings can take place in non-initial positions; the strings could be part of phrases of meaningful sentences, instead of just a imperative phrase redirecting readers to other news articles)
2. The strings "iklan" and "advertisement" seem to only be noises when the words are all capitalized

#### To account for this problem, we will:
1. Modify the regex to only filter out the strings "iklan" and "advertisement" when they are all capitalized
2. Use spaCy to segment the news into separate sentences, then modify the regex expression to filter out "baca juga" in sentence initial positions

## 2. Updating regular expressions

In [74]:
# new regex expression, where "iklan" and "advertisement" are only retrieved when in uppercase
new_regex = r"(?i)\b(baca juga)\b|\b(ADVERTISEMENT|IKLAN)\b"

new_dataset = news_dataset_clean

# add new column matches where True indicates instances of news text with unecessary word
new_dataset['matches'] = new_dataset['content'].str.contains(new_regex)

# count the no. of True observation of column 'matches' and len of news_dataset_clean
new_nrow_matches = new_dataset['matches'].sum()
new_nrow_news_dataset = len(new_dataset)

print(f"{new_nrow_matches} instances of news text with unecessary words \n{new_nrow_matches/new_nrow_news_dataset:.2f} percent of instances with unecessary words")

310 instances of news text with unecessary words 
0.34 percent of instances with unecessary words


In [75]:
# the keywords inside the regex for loop
new_unecessary_words = ["baca juga",  "ADVERTISEMENT", "IKLAN"]

# looping each keyword in key_unecessary_words
for keyword in new_unecessary_words:
    if keyword == "baca juga":
        pattern = r"(?i)\b" + re.escape(keyword) + r"\b"  # case-insensitive for "baca juga"
    else:
        pattern = r"\b" + re.escape(keyword) + r"\b"  # exact match for "ADVERTISEMENT" and "IKLAN"
    
    new_matches = new_dataset[new_dataset['content'].str.contains(pattern, na=False)] # subset dataframe containing keyword
    new_matches = new_matches.reset_index(drop=True) # reset the indexing
    print(f"--- Unecessary words: {keyword} ({len(new_matches)} observations) ---")
    
    if len(new_matches) > 0:
        for num in range(min(3, len(new_matches))):
            random_news_instance = random.randint(0, len(new_matches) - 1)
            print(f"Random news {num}: {new_matches['content'].iloc[random_news_instance]}\n")
    else:
        print(f"No matches for {keyword} found in the dataset.\n")

--- Unecessary words: baca juga (291 observations) ---
Random news 0: Ambil Saldo DANA Gratis Sekarang! Klaim Link Tautan Bernilai Rp 121 Ribu Cair Langsung Ke Rekening Reporter: Dian Saptari | Editor: Dian Saptari | Rabu 19-03-2025,17:00 WIB Dapatkan saldo gratis hari ini. Sumber Foto. Dian Saptari. Radar Lampung -- RADARLAMPUNG.CO.ID - Segera ambil saldo DANA gratis sekarang lewat klaim link tautan bernilai Rp 121 ribu hari ini. Cair langsung ke rekening dapatkan segera saldo DANA gratis tanpa syarat. Saldo gratis dana dari link DANA Kaget dapat di klaim hari ini. Cukup modal hp dan lakukan klik tautan link DANA Kaget cairkan saldonya. BACA JUGA: Mutasi Polri Maret 2025, 8 Perwira Tinggi Promosi Jadi Jenderal Bintang Dua BACA JUGA: Mutasi Polri Maret 2025, Puluhan Perwira Menengah Promosi Jadi Kapolres Pengguna cukup siapkan nomor akun e wallet untuk terima transferan saldo dana yang berhasil di klaim. Berikut cara aktifkan nomor akun e wallet sebagai bukti pencairan saldo gratis har

#### The modification of the regex has decreased the number of observation of text with "iklan" and "advertisement" strings. It also showed that some the string "advertisement" are sentence-bound. 

#### To conclude, from these strings: "baca juga", "advertisement", and "iklan", two are likely to be sentence-bound: "baca juga" and "advertisement". Therefore, we will use spaCy library to work with these strings.

#### As of the other strings: the non-sentence-bound "iklan" and "advertisement" in uppercase, we will use regex to clean those strings from news texts.

## 3. Sentence segmentation/tokenization with spaCy

In [76]:
import spacy

nlp = spacy.load("xx_ent_wiki_sm")
sentencizer = nlp.add_pipe('sentencizer')
sentencizer.from_disk(f"{os.getcwd()}/sentencizer.json") # json file with list of punctuation mark

text = "The U.S. Drug Enforcement Administration (DEA) says hello. And have a nice day. Halo! Berikut merupakan teks berbahasa Indonesia. Presiden A.S. baru saja memberikan pidato di depan Gedung Putih."

text_unpunctuated = "She loves to read books on weekends she enjoys visiting the library and discovering new authors he prefers to read mystery novels and often finishes one in a day their favorite place is the bookstore which is near the park they meet there every Saturday morning to share their new finds they sometimes discuss their favorite characters and plot twists with excitement"

# Process the text using spaCy
doc = nlp(text)

# Print the segmented sentences
for sent in doc.sents:
    print(sent.text)

# Process the text using spaCy
doc = nlp(text_unpunctuated)

# Print the segmented sentences
for sent in doc.sents:
    print(sent.text)

The U.S. Drug Enforcement Administration (DEA) says hello.
And have a nice day.
Halo!
Berikut merupakan teks berbahasa Indonesia.
Presiden A.S. baru saja memberikan pidato di depan Gedung Putih.
She loves to read books on weekends she enjoys visiting the library and discovering new authors he prefers to read mystery novels and often finishes one in a day their favorite place is the bookstore which is near the park they meet there every Saturday morning to share their new finds they sometimes discuss their favorite characters and plot twists with excitement


#### The spaCy setencizer pipe works by segmenting sentences delimited by punctuation marks in a smart way. Unlike regex which is more exact in nature, spaCy setencizer takes into consideration abbreviations, such as "A.S." and "U.S." It can handle both Indonesian and English

#### Caveat: The spaCy setencizer pipe does not work well with unpunctuated text. We could create a function to make rough estimates of sentences in unpunctuated text, however it does not retrieve acccurate results.

## 4. Sentence segmentation with NLTK

In [77]:
import nltk

text = "The U.S. Drug Enforcement Administration (DEA) says hello. And have a nice day. Halo! Berikut merupakan teks berbahasa Indonesia. Presiden A.S. baru saja memberikan pidato di depan Gedung Putih."

text_unpunctuated = "She loves to read books on weekends she enjoys visiting the library and discovering new authors he prefers to read mystery novels and often finishes one in a day their favorite place is the bookstore which is near the park they meet there every Saturday morning to share their new finds they sometimes discuss their favorite characters and plot twists with excitement"

tokenized_text = nltk.tokenize.sent_tokenize(text)

for token in tokenized_text:
    print(token)

tokenized_text = nltk.tokenize.sent_tokenize(text_unpunctuated)

for token in tokenized_text:
    print(token)

The U.S. Drug Enforcement Administration (DEA) says hello.
And have a nice day.
Halo!
Berikut merupakan teks berbahasa Indonesia.
Presiden A.S. baru saja memberikan pidato di depan Gedung Putih.
She loves to read books on weekends she enjoys visiting the library and discovering new authors he prefers to read mystery novels and often finishes one in a day their favorite place is the bookstore which is near the park they meet there every Saturday morning to share their new finds they sometimes discuss their favorite characters and plot twists with excitement


#### Just like spaCy sentencizer pipe, the NLTK tokenize method also works by segmenting sentences delimited by punctuation marks in a smart way. It can also handle both Indonesian and English

#### Caveat: Just like the spaCy setencizer pipe, it does not work well with unpunctuated text.

#### The challenge of sentence segmentation from both models come from the fact that they cannot detect sentences in unpunctuated text. Meanwhile, most of "baca juga" string sentences are not delimited by any punctuation marks. 

## 5. Heuristic approach to text data cleaning

#### From previous observation, we know that the strings "baca juga" and "advertisement" are sentence bound. Since spaCy and NLTK segment sentence based on a hybrid nature and it cannot segment sentences in an unpunctuated text, we need to move to a heuristic approach. We need to understand the patterns of the sentences in which the strings "baca juga" and "advertisement" are bound to, so that we can develop an algorithm that cleans them from text data. For this, we will be analyzing the patterns of sentences bound to aforementioned strings. 

### 5.1. Analyzing sentence pattern bound with "baca juga" string

#### We will be parsing the 25 token of strings after "baca juga" string to observe the pattern of sentence bound with "baca juga" string.

In [176]:
contains_baca_juga = r"(?i)\b(baca juga)\b"

baca_juga_dataset = new_dataset[new_dataset['content'].str.contains(contains_baca_juga, na=False)]
baca_juga_dataset = baca_juga_dataset.reset_index(drop=True)

right_context_baca_juga = re.compile(r"(?i)(baca juga[:\-\s]+(?:[\w\W]+?)(?=\s+baca juga|$))")

# take 20 random news articles, print every instances of "baca juga" strings and its right context 
for num in range(20):
    print(f"--- Random news {num} ---")

    random_news_instance = random.randint(0, len(baca_juga_dataset) - 1)
    text = baca_juga_dataset['content'][random_news_instance]
    matches = right_context_baca_juga.finditer(text)
    for i, match in enumerate(matches, start=1):
        match_text = match.group(1).strip()  
        words = match_text.split()
        first_50_words = words[:50]
        
        print(f"Random text {i}: {' '.join(first_50_words)}\n")
    
    print("\n")

--- Random news 0 ---
Random text 1: BACA JUGA: Mau Bisnis Lancar? Gunakan Dompet Digital untuk Pedagang dan Nikmati Keuntungannya!

Random text 2: BACA JUGA: Mengenal Keamanan Dompet Digital Blockchain yang Menjaga Transaksi Anda Tetap Aman Tidak hanya itu, banyak marketplace yang bekerja sama dengan dompet digital untuk menghadirkan promo khusus. Beli bahan makanan atau kebutuhan dapur pun jadi lebih terjangkau. Bahkan beberapa penyedia dompet digital menawarkan gratis ongkir untuk pembelian makanan online.

Random text 3: BACA JUGA: Kenali Dompet Digital Sakuku, Solusi Praktis untuk Cashless Society

Random text 4: BACA JUGA: Dompet Digital, Cara Praktis Kelola Keuangan di Era Modern GoPay sering memberikan diskon untuk pembelian makanan di layanan ojek online. Sementara OVO dan ShopeePay banyak digunakan untuk pembayaran di restoran dan marketplace. DANA memiliki berbagai program cashback yang bisa dimanfaatkan untuk belanja kebutuhan buka puasa. Sedangkan LinkAja sangat

Random te

#### We can observe that:
1. 
2. 
3. 